In [1]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.figures.figures as figs
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh
from entropy import config

sns.set_style("whitegrid")
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [33]:
df = hd.read_sample("XX7")
hd.inspect(df)

Time for read_sample    : 3.24 minutes
(7,531,116, 31)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,entropy_sptac
0,19510816,2014-03-07,107,9.20,paypal sandrpowell 9001,paypal,spend,services,0.0,gl3 4,2010-05-10,40k to 50k,1984.0,2014-05-11,238820,2020-08-16 12:13:00,barclaycard,credit card,2014-07-18,2017-09-12,True,NaN,paypal,enjoyment,home electronics,home electronics,u,201403,NaN,27530.926758,2.036569
1,19510817,2014-03-08,107,6.94,paypal smartbitsso 9001,paypal,spend,services,0.0,gl3 4,2010-05-10,40k to 50k,1984.0,2014-05-11,238820,2020-08-16 12:13:00,barclaycard,credit card,2014-07-18,2017-09-12,True,NaN,paypal,enjoyment,home electronics,home electronics,u,201403,NaN,27530.926758,2.036569


In [68]:
df.groupby("user_id").ym.nunique().describe()

count    2716.000000
mean       29.442563
std        16.319621
min         4.000000
25%        17.000000
50%        26.000000
75%        40.000000
max       102.000000
Name: ym, dtype: float64

In [37]:
%%timeit

mask = df.account_type.eq("current")
users = df[mask].user_id.unique()
df[df.user_id.isin(users)]

1.77 s ± 252 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
df

,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,entropy_sptac
0,19510816,2014-03-07,107,9.200000,paypal sandrpowell 9001,paypal,spend,services,0.0,gl3 4,2010-05-10,40k to 50k,1984.0,2014-05-11,238820,2020-08-16 12:13:00,barclaycard,credit card,2014-07-18,2017-09-12,True,NaN,paypal,enjoyment,home electronics,home electronics,u,201403,NaN,27530.926758,2.036569
1,19510817,2014-03-08,107,6.940000,paypal smartbitsso 9001,paypal,spend,services,0.0,gl3 4,2010-05-10,40k to 50k,1984.0,2014-05-11,238820,2020-08-16 12:13:00,barclaycard,credit card,2014-07-18,2017-09-12,True,NaN,paypal,enjoyment,home electronics,home electronics,u,201403,NaN,27530.926758,2.036569
2,19510818,2014-03-09,107,6.020000,paypal e2010cv 9001,paypal,spend,services,0.0,gl3 4,2010-05-10,40k to 50k,1984.0,2014-05-11,238820,2020-08-16 12:13:00,barclaycard,credit card,2014-07-18,2017-09-12,True,NaN,paypal,enjoyment,home electronics,home electronics,u,201403,NaN,27530.926758,2.036569
3,19510819,2014-03-09,107,6.490000,paypal michaelmick 9001,paypal,spend,services,0.0,gl3 4,2010-05-10,40k to 50k,1984.0,2014-05-11,238820,2020-08-16 12:13:00,barclaycard,credit card,2014-07-18,2017-09-12,True,NaN,paypal,enjoyment,home electronics,home electronics,u,201403,NaN,27530.926758,2.036569
4,19510820,2014-03-11,107,38.000000,paypal proinnovati 9001,paypal,spend,services,0.0,gl3 4,2010-05-10,40k to 50k,1984.0,2014-05-11,238820,2020-08-16 12:13:00,barclaycard,credit card,2014-07-18,2017-09-12,True,NaN,paypal,enjoyment,home electronics,home electronics,u,201403,NaN,27530.926758,2.036569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7531111,806482551,2020-07-27,589617,-1000.000000,mdbremoved,NaN,NaN,NaN,0.0,e15 2,2020-08-12,30k to 40k,1988.0,2020-08-12,1732215,2020-08-16 02:27:00,natwest bank,current,2020-08-13,1900-01-01,False,2816.600098,NaN,NaN,NaN,NaN,c,202007,1117.979736,32763.016741,2.864084
7531112,806482543,2020-07-28,589617,-2544.020020,mdbremoved,NaN,income,earnings,0.0,e15 2,2020-08-12,30k to 40k,1988.0,2020-08-12,1732215,2020-08-16 02:27:00,natwest bank,current,2020-08-13,2020-08-14,False,2816.600098,NaN,salary or wages (main),NaN,NaN,u,202007,2826.399902,32763.016741,2.864084
7531113,806482542,2020-07-28,589617,1000.000000,mdbremoved via mobile pymt fp 28 07 20 10 000n,NaN,NaN,NaN,0.0,e15 2,2020-08-12,30k to 40k,1988.0,2020-08-12,1732215,2020-08-16 02:27:00,natwest bank,current,2020-08-13,1900-01-01,True,2816.600098,non merchant mbl,NaN,NaN,NaN,c,202007,2826.399902,32763.016741,2.864084
7531114,806482544,2020-07-28,589617,-164.399994,mdbremoved,NaN,spend,retail,0.0,e15 2,2020-08-12,30k to 40k,1988.0,2020-08-12,1732215,2020-08-16 02:27:00,natwest bank,current,2020-08-13,1900-01-01,False,2816.600098,NaN,refunded purchase,NaN,refunded purchase,c,202007,2826.399902,32763.016741,2.864084
